# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# import libraries
from sqlalchemy import create_engine
import sqlite3

import nltk
nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_recall_fscore_support

[nltk_data] Downloading package punkt to /Users/dwadhaw/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dwadhaw/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/dwadhaw/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/dwadhaw/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
# load data from database
conn = sqlite3.connect('../data/DisasterResponse.db')
df = pd.read_sql('SELECT * FROM DisasterTable', conn)

X = df[['message']]
y = df[df.columns.difference(['id', 'message', 'original', 'genre'])]

### 2. Write a tokenization function to process your text data

In [12]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    stop_words = stopwords.words('english')
    # lemmatize and remove stop words
    #tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        if clean_tok not in stop_words:
            clean_tokens.append(clean_tok)

    return clean_tokens

In [49]:
def display_results(y_test, y_pred):
    
    for category in y_test.columns.values:
        print('**Processing {} comments...**'.format(category))
    
        # calculating test accuracy
        print("Precision            Recall               fScore")
        print(precision_recall_fscore_support(y_test[category], y_pred[category], average='weighted'))
        print("\n")
    #    m = MultiLabelBinarizer().fit(y_test.to_numpy())
#   print(precision_recall_fscore_support(m.transform(y_test.to_numpy()), m.transform(y_pred), average='weighted'))

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
def old_main():
    X_train, X_test, y_train, y_test = train_test_split(X['message'], y, random_state=1)
    
    forest = RandomForestClassifier(n_estimators=10, random_state=1)
    
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf' , MultiOutputClassifier(forest))
    ])

    # train classifier
    pipeline.fit(X_train, y_train)

    # predict on test data
    y_pred = pipeline.predict(X_test)

    # display results
    display_results(y_test,y_pred)

In [33]:
def old_main_2():
    X_train, X_test, y_train, y_test = train_test_split(X['message'], y, random_state=1)
    
    from sklearn.feature_extraction.text import TfidfVectorizer
    
    vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
    vectorizer.fit(X_train)
    vectorizer.fit(X_test)
    vectorizer.fit(y_train)
    vectorizer.fit(y_test)
    
    X_train = vectorizer.transform(X_train)
    X_test = vectorizer.transform(X_test)
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.pipeline import Pipeline
    from sklearn.metrics import accuracy_score
    from sklearn.multiclass import OneVsRestClassifier
    
    # Using pipeline for applying logistic regression and one vs rest classifier
    LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])
    for category in y.columns.values:
        print('**Processing {} comments...**'.format(category))
    
        # Training logistic regression model on train data
        LogReg_pipeline.fit(X_train, y_train[category])
    
        # calculating test accuracy
        prediction = LogReg_pipeline.predict(X_test)
        print('Test accuracy is {}'.format(accuracy_score(y_test[category], prediction)))
        print("\n")

In [38]:
def old_main_3():
    X_train, X_test, y_train, y_test = train_test_split(X['message'], y, random_state=1)
    
    forest = RandomForestClassifier()
    
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf' , MultiOutputClassifier(forest))
    ])

    # train classifier
    pipeline.fit(X_train, y_train)

    # predict on test data
    y_pred = pipeline.predict(X_test)

    # display results
    for category in y.columns.values:
        print('**Processing {} comments...**'.format(category))
        display_results(y_test[category], y_pred[category])
        print("\n")

In [30]:
def main():
    X_train, X_test, y_train, y_test = train_test_split(X['message'], y, random_state=1)
    
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.linear_model import LogisticRegression
    from sklearn.pipeline import Pipeline
    from sklearn.metrics import accuracy_score
    from sklearn.multiclass import OneVsRestClassifier
    
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf' , OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1))
    ]) 
    
    
    # Using pipeline for applying logistic regression and one vs rest classifier
    
    for category in y.columns.values:
        print('**Processing {} comments...**'.format(category))
    
        # Training logistic regression model on train data
        pipeline.fit(X_train, y_train[category])
    
        # calculating test accuracy
        prediction = pipeline.predict(X_test)
        print('Test accuracy is {}'.format(accuracy_score(y_test[category], prediction)))
        display_results(y_test[category], prediction)
        print("\n")

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X['message'], y, random_state=1)
    
forest = RandomForestClassifier(n_estimators=10, random_state=1)
    
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf' , MultiOutputClassifier(forest))
])

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

# display results
#display_results(y_test,y_pred)

**Processing aid_centers comments...**
Precision            Recall               fScore
(0.9784495622906411, 0.9890143423863289, 0.9837035872492437, None)


**Processing aid_related comments...**
Precision            Recall               fScore
(0.7449033792459936, 0.7451937747940189, 0.7397103339558325, None)


**Processing buildings comments...**
Precision            Recall               fScore
(0.9405143273203443, 0.9499542264266096, 0.9322359819843368, None)


**Processing child_alone comments...**
Precision            Recall               fScore
(1.0, 1.0, 1.0, None)


**Processing clothing comments...**
Precision            Recall               fScore
(0.9844257309125398, 0.9862679279829112, 0.9810412286397538, None)


**Processing cold comments...**
Precision            Recall               fScore
(0.9770596455933365, 0.9827586206896551, 0.9769299303671279, None)


**Processing death comments...**
Precision            Recall               fScore
(0.9528827188714365, 0.9601769911

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(precision_recall_fscore_support(y_test, y_pred, average='weighted'))

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.